<a href="https://colab.research.google.com/github/INT-Group-9/ColabNotebook/blob/Jake/cifa10_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms


In [11]:
train = datasets.CIFAR10("./data", train=True, download = True,
                         transform = transforms.Compose([transforms.ToTensor()]))

test = datasets.CIFAR10("./data", train=False, download=True,
                         transform = transforms.Compose([transforms.ToTensor()]))

Files already downloaded and verified
Files already downloaded and verified


In [12]:
trainset = torch.utils.data.DataLoader(train, batch_size=10, shuffle=True)
testSet = torch.utils.data.DataLoader(test, batch_size=10, shuffle=True)

In [13]:
class Net(nn.Module):

  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride = 1, padding = 1)
    self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, stride = 1, padding = 1)
    self.conv3 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, stride = 1, padding = 1)
    self.conv4 = nn.Conv2d(in_channels=12, out_channels=3, kernel_size=3, stride = 1, padding = 1)
    self.fc1 = nn.Linear(in_features = 32*32*3, out_features=10)
  
  def forward(self, input):
    output = F.relu(self.conv1(input))
    output = F.relu(self.conv2(output))
    output = F.relu(self.conv3(output))
    output = F.relu(self.conv4(output))

    output = self.fc1(output)


    return F.log_softmax(output, dim=1)

    
net = Net()
print(net)


Net(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(12, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=3072, out_features=10, bias=True)
)
